In [1]:
from astropy.io import fits
from photutils.detection import DAOStarFinder, IRAFStarFinder
from photutils.psf import DAOGroup
from photutils.psf import IntegratedGaussianPRF
from photutils.background import MMMBackground
from photutils.background import MADStdBackgroundRMS
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.stats import gaussian_sigma_to_fwhm
import reduce_20161118 as red


sigma_psf = 2.0 #not sure what to use for this...
image = fits.getdata("open_red/obj_135_red.fits", header=False)

bkgrms = MADStdBackgroundRMS()

std = bkgrms(image)

iraffind = IRAFStarFinder(threshold = 3.5*std, fwhm=sigma_psf*gaussian_sigma_to_fwhm, minsep_fwhm=0.01, roundhi=5.0, roundlo=-5.0, sharplo=0.0, sharphi=2.0)
daogroup = DAOGroup(2.0*sigma_psf*gaussian_sigma_to_fwhm)

mmm_bkg = MMMBackground()

psf_model = IntegratedGaussianPRF(sigma=sigma_psf)

fitter = LevMarLSQFitter()

In [ ]:
image_bin = red.rebin(image, 10)

from photutils.psf import IterativelySubtractedPSFPhotometry

photometry = IterativelySubtractedPSFPhotometry(finder=iraffind, group_maker=daogroup, bkg_estimator=mmm_bkg, psf_model=psf_model, fitter=LevMarLSQFitter(),niters=2, fitshape=(11,11))

result_tab = photometry(image_bin)
residual_image = photometry.get_residual_image()

In [21]:
result_tab.sort('flux_0')
result_tab

x_0,y_0,flux_0,id,group_id,iter_detected,x_fit,y_fit,flux_fit
float64,float64,float64,int32,int32,int64,float64,float64,float64
193.196111423,168.816439906,-354871.183991,156,18,2,186.946287437,169.089727624,53240.9986647
187.945585191,166.561537972,-198342.115161,147,18,2,205.90355333,173.681530281,-29783.4946447
188.019061707,171.497657048,-190325.780515,167,18,2,193.938584854,171.844790114,-158285.543807
190.480856932,173.985215981,-188857.357509,145,20,2,265.143350311,151.768066545,-15086.066421
194.139741158,162.979026679,-178885.306726,137,17,2,156.527406658,148.644089473,-7323.61575388
198.966591489,170.196605014,-175365.836076,157,18,2,193.543693232,169.034803089,750000.147836
192.973133352,174.977825896,-155988.53145,161,22,2,45.6131875125,168.315335334,-1844.67092476
187.027720071,169.021231932,-154665.973203,154,18,2,191.313456353,167.700424473,-266781.51275
78.5998056228,50.9878991139,-37993.2574433,128,1,2,0.593166955143,141.375109029,247.39434352


In [10]:
original_plate_scale = 23.0 #mas/pixel
plate_scale_arcsec = original_plate_scale * 3 * 10 * 1e-3 #as/pix, (binned twice)

In [2]:
from photutils import CircularAperture
from photutils import aperture_photometry

radius_arcsec = 0.4
radius_pixel = radius_arcsec / plate_scale_arcsec
positions = [(result_tab[-1][0], (result_tab[-1][1]))]
apertures = CircularAperture(positions, r=radius_pixel)

data = image

phot_table = aperture_photometry(data, apertures)
print(phot_table)